In [1]:
import os
import pandas as pd
from db_insert_dohy import MySQLInserter

In [2]:
# get information from environment variables
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = 'localhost'
port = 3306
database = 'monetary_policy'

In [3]:
# connect to base rate 
df_base = pd.read_json('base_rate.json')
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      55 non-null     int64  
 1   날짜      55 non-null     object 
 2   기준금리    55 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.4+ KB


In [13]:
df_base['date'] = df_base['연도'].astype(str) + '-' + df_base['날짜'].str.replace('월 ', '-').str.replace('일', '')
df_base['date'] = pd.to_datetime(df_base['date'])
df_base_insert = df_base[['기준금리', 'date']]
df_base_insert = df_base_insert.rename(columns={'기준금리': 'base_rate', 'date': 'rate_announcement_date'})
df_base_sorted = df_base_insert.sort_values(by='rate_announcement_date')
df_base_sorted.head()


,base_rate,rate_announcement_date
54,4.75,1999-05-06
53,5.00,2000-02-10
52,5.25,2000-10-05
51,5.00,2001-02-08
50,4.75,2001-07-05


In [15]:
inserter = MySQLInserter(user, password, host, port, database)
inserter.insert_many('base_rate', df_base_sorted)
inserter.close()

In [8]:
# connect to call rate
df_call = pd.read_csv('../preprocess/data_combine/call_rates_3.csv')
df_call.head()

,Unnamed: 0,Date,Call_Rate,CD,Treasury_Bond_3Y,Treasury_Bond_5Y,Corporate_Bond
0,0,2014.08.11,2.49,2.63,2.51,2.73,2.91
1,1,2014.08.12,2.49,2.62,2.52,2.75,2.91
2,2,2014.08.13,2.49,2.61,2.52,2.75,2.91
3,3,2014.08.14,2.49,2.46,2.54,2.76,2.93
4,4,2014.08.18,2.24,2.43,2.56,2.79,2.95


In [11]:
df_call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2466 non-null   int64  
 1   Date              2466 non-null   object 
 2   Call_Rate         2466 non-null   float64
 3   CD                2466 non-null   object 
 4   Treasury_Bond_3Y  2466 non-null   object 
 5   Treasury_Bond_5Y  2466 non-null   object 
 6   Corporate_Bond    2466 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 135.0+ KB


In [16]:
df_call['Date'] = pd.to_datetime(df_call['Date'])
df_call.rename(columns={'Call Rate': 'call_rate', 'Date': 'rate_announcement_date'}, inplace=True)